# Interactive Camera Analysis for RenderMe360 s3_all

This notebook provides interactive tools for exploring camera configurations and testing different subset selections.

In [ ]:
# Import required libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import json
from IPython.display import display, Image, HTML
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D

# Add script directory to path
sys.path.append('/ssd2/zhuoyuan/renderme360_temp/download_all/process_data_scripts')
from analyze_s3_cameras_dynamic import DynamicCameraAnalyzer

print("Libraries loaded successfully!")

## 1. Load and Analyze Camera Data

In [ ]:
# Define subject directories
base_dir = Path('/ssd2/zhuoyuan/renderme360_temp/download_all/subjects')
subjects = ['0018', '0019', '0026']
subject_dirs = [base_dir / subj for subj in subjects]

# Create analyzer
print("Analyzing camera data...")
analyzer = DynamicCameraAnalyzer(subject_dirs)

# Display summary
print(f"\nAnalysis complete!")
print(f"Subjects analyzed: {len(analyzer.subjects_data)}")
print(f"Common cameras across all subjects: {len(analyzer.common_cameras)}")
print(f"\nCommon camera IDs: {analyzer.common_cameras}")

## 2. Camera Position Statistics

In [ ]:
# Get metrics from first subject
subject_id = list(analyzer.subjects_data.keys())[0]
metrics = analyzer.subjects_data[subject_id]['metrics']

# Create DataFrame for easier analysis
metrics_df = pd.DataFrame([
    {
        'Camera ID': cam_id,
        'X (m)': m['position'][0],
        'Y (m)': m['position'][1],
        'Z (m)': m['position'][2],
        'Distance (m)': m['distance'],
        'Azimuth (°)': m['azimuth_deg'],
        'Elevation (°)': m['elevation_deg']
    }
    for cam_id, m in metrics.items()
])

# Display statistics
print("Camera Position Statistics:")
print("="*50)
display(metrics_df.describe())

# Show first few cameras
print("\nFirst 10 cameras:")
display(metrics_df.head(10))

## 3. Interactive 3D Camera Visualization

In [ ]:
%matplotlib widget

# Create interactive 3D plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Get positions
positions = np.array([metrics[cam_id]['position'] for cam_id in sorted(metrics.keys())])
cam_ids = sorted(metrics.keys())

# Plot cameras
scatter = ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2],
                    c=positions[:, 2], cmap='viridis', s=100, alpha=0.6)

# Add labels
for i, cam_id in enumerate(cam_ids):
    ax.text(positions[i, 0], positions[i, 1], positions[i, 2],
            str(cam_id), fontsize=8)

# Add subject at origin
ax.scatter([0], [0], [0], color='red', s=200, marker='*', label='Subject')

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title('Interactive 3D Camera Positions')
ax.legend()

plt.colorbar(scatter, ax=ax, label='Height (m)', pad=0.1)
plt.show()

## 4. Coverage Gap Analysis

In [ ]:
# Analyze coverage gaps
gaps = analyzer.analyze_coverage_gaps()

# Create DataFrame
gaps_df = pd.DataFrame(gaps)

print("Coverage Gap Analysis:")
print("="*50)
print(f"Average gap: {np.mean(gaps_df['gap_degrees']):.1f}°")
print(f"Ideal gap for {len(metrics)} cameras: {360/len(metrics):.1f}°")
print(f"\nLargest gaps:")
display(gaps_df.head(10))

# Plot gap distribution
plt.figure(figsize=(10, 4))
plt.hist(gaps_df['gap_degrees'], bins=20, edgecolor='black', alpha=0.7)
plt.axvline(x=np.mean(gaps_df['gap_degrees']), color='red', linestyle='--', label='Average')
plt.xlabel('Gap Size (degrees)')
plt.ylabel('Count')
plt.title('Distribution of Angular Gaps Between Cameras')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Interactive Camera Subset Selection

In [ ]:
# Create interactive widget for subset selection
%matplotlib inline

def visualize_subset(num_cameras, show_labels=True):
    """Visualize a camera subset selection."""
    suggestions = analyzer.suggest_optimal_subsets([num_cameras])
    
    if num_cameras not in suggestions:
        print(f"Cannot create subset of {num_cameras} cameras")
        return
    
    subset = suggestions[num_cameras]
    selected_cams = subset['cameras']
    
    # Create figure with subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 1. Top-down view
    ax1 = axes[0]
    for cam_id, m in metrics.items():
        pos = m['position']
        if cam_id in selected_cams:
            ax1.scatter(pos[0], pos[1], color='red', s=100, alpha=0.8)
            if show_labels:
                ax1.annotate(str(cam_id), (pos[0], pos[1]), fontsize=8, fontweight='bold')
        else:
            ax1.scatter(pos[0], pos[1], color='lightgray', s=30, alpha=0.3)
    
    ax1.scatter(0, 0, color='blue', s=200, marker='*')
    ax1.set_xlabel('X (m)')
    ax1.set_ylabel('Y (m)')
    ax1.set_title('Top-Down View')
    ax1.axis('equal')
    ax1.grid(True, alpha=0.3)
    
    # 2. Polar view
    ax2 = plt.subplot(132, projection='polar')
    for cam_id, m in metrics.items():
        if cam_id in selected_cams:
            ax2.scatter(m['azimuth_rad'], m['distance'], color='red', s=100, alpha=0.8)
            if show_labels:
                ax2.annotate(str(cam_id), (m['azimuth_rad'], m['distance']), 
                           fontsize=8, fontweight='bold')
        else:
            ax2.scatter(m['azimuth_rad'], m['distance'], color='lightgray', s=30, alpha=0.3)
    ax2.set_title('Angular Distribution')
    
    # 3. Metrics display
    ax3 = axes[2]
    ax3.axis('off')
    
    # Display metrics as text
    metrics_text = f"""Camera Subset Metrics
    {'='*25}
    Cameras: {num_cameras}
    Selected IDs: {selected_cams}
    
    Coverage Metrics:
    - Average gap: {subset['avg_gap']:.1f}°
    - Max gap: {subset['max_gap']:.1f}°
    - Min gap: {subset['min_gap']:.1f}°
    - Gap std: {subset['gap_std']:.1f}°
    
    Quality Score: {subset['quality_score']:.1f}/100
    
    Storage Estimates:
    - Per subject: {subset['storage_per_subject_gb']:.1f} GB
    - 500 subjects: {subset['storage_500_subjects_gb']:.0f} GB
    """
    
    ax3.text(0.1, 0.5, metrics_text, transform=ax3.transAxes, 
            fontsize=10, verticalalignment='center', fontfamily='monospace')
    
    plt.suptitle(f'Camera Subset Visualization - {num_cameras} Cameras', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    return selected_cams

# Create interactive widget
camera_slider = widgets.IntSlider(
    value=12,
    min=4,
    max=20,
    step=2,
    description='Cameras:',
    continuous_update=False
)

labels_checkbox = widgets.Checkbox(
    value=True,
    description='Show Labels'
)

widgets.interact(visualize_subset, num_cameras=camera_slider, show_labels=labels_checkbox);

## 6. Compare Different Subset Strategies

In [ ]:
# Compare different subset sizes
subset_sizes = [4, 6, 8, 10, 12, 16, 20]
suggestions = analyzer.suggest_optimal_subsets(subset_sizes)

# Create comparison DataFrame
comparison_data = []
for size, data in suggestions.items():
    comparison_data.append({
        'Cameras': size,
        'Avg Gap (°)': f"{data['avg_gap']:.1f}",
        'Max Gap (°)': f"{data['max_gap']:.1f}",
        'Quality Score': f"{data['quality_score']:.1f}",
        'Storage/Subject (GB)': f"{data['storage_per_subject_gb']:.1f}",
        'Storage/500 (TB)': f"{data['storage_500_subjects_gb']/1024:.1f}"
    })

comparison_df = pd.DataFrame(comparison_data)

print("Camera Subset Comparison:")
print("="*70)
display(comparison_df)

# Plot storage vs quality
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Storage plot
storage_values = [suggestions[s]['storage_500_subjects_gb']/1024 for s in subset_sizes]
ax1.plot(subset_sizes, storage_values, 'o-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Cameras')
ax1.set_ylabel('Storage for 500 Subjects (TB)')
ax1.set_title('Storage Requirements')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='1TB limit')
ax1.axhline(y=2, color='orange', linestyle='--', alpha=0.5, label='2TB limit')
ax1.legend()

# Quality plot
quality_values = [suggestions[s]['quality_score'] for s in subset_sizes]
ax2.plot(subset_sizes, quality_values, 'o-', linewidth=2, markersize=8, color='green')
ax2.set_xlabel('Number of Cameras')
ax2.set_ylabel('Quality Score')
ax2.set_title('Coverage Quality')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Generate Custom Configuration

In [ ]:
def generate_config_yaml(num_cameras):
    """Generate config.yaml snippet for a specific camera count."""
    suggestions = analyzer.suggest_optimal_subsets([num_cameras])
    
    if num_cameras not in suggestions:
        print(f"Cannot generate config for {num_cameras} cameras")
        return
    
    selected_cameras = suggestions[num_cameras]['cameras']
    
    config = f"""# Configuration for {num_cameras} cameras in s3_all
# Quality Score: {suggestions[num_cameras]['quality_score']:.1f}/100
# Storage: {suggestions[num_cameras]['storage_per_subject_gb']:.1f} GB per subject
# Total for 500 subjects: {suggestions[num_cameras]['storage_500_subjects_gb']:.0f} GB

extraction:
  # Camera selection for s3_all
  cameras: {selected_cameras}
  
  # Extract only s3_all with these cameras
  performances:
    - "s3_all"
  
  # Data modalities to extract
  modalities:
    - "metadata"
    - "calibration"
    - "images"
    - "audio"

# For single-view performances, run separately with:
# cameras: [25]
# performances: ["s1_all", "s2_all", "s4_all", "s5_all", "s6_all"]
"""
    
    print(config)
    
    # Also save to file
    output_file = f"config_s3all_{num_cameras}_cameras.yaml"
    with open(output_file, 'w') as f:
        f.write(config)
    print(f"\nConfiguration saved to: {output_file}")

# Interactive widget for config generation
config_slider = widgets.IntSlider(
    value=12,
    min=4,
    max=20,
    step=2,
    description='Cameras:',
    style={'description_width': 'initial'}
)

button = widgets.Button(description="Generate Config")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        generate_config_yaml(config_slider.value)

button.on_click(on_button_clicked)

display(widgets.VBox([config_slider, button, output]))

## 8. Export Analysis Results

In [ ]:
# Export all camera positions to CSV
output_dir = Path('/ssd2/zhuoyuan/renderme360_temp/download_all/visualizations/camera_analysis')
output_dir.mkdir(parents=True, exist_ok=True)

# Save camera metrics
metrics_df.to_csv(output_dir / 'camera_metrics.csv', index=False)
print(f"Camera metrics saved to: {output_dir / 'camera_metrics.csv'}")

# Save gap analysis
gaps_df.to_csv(output_dir / 'coverage_gaps.csv', index=False)
print(f"Coverage gaps saved to: {output_dir / 'coverage_gaps.csv'}")

# Save subset recommendations
comparison_df.to_csv(output_dir / 'subset_recommendations.csv', index=False)
print(f"Subset recommendations saved to: {output_dir / 'subset_recommendations.csv'}")

print("\nAll analysis results exported successfully!")

## Summary

This notebook provides interactive tools for:
1. Analyzing camera positions and coverage
2. Testing different subset configurations
3. Generating configuration files
4. Visualizing camera arrangements

### Key Findings:
- All subjects have the same 38 cameras in s3_all
- Camera distribution has some irregularities (large gaps)
- 12 cameras provides good balance of quality and storage
- Different subset sizes suit different research needs

### Next Steps:
1. Choose appropriate subset size based on storage constraints
2. Update config.yaml with selected cameras
3. Run extraction for remaining subjects
4. Monitor storage usage during extraction